In [1]:
from query_deepseek import query_deepseek
from query_gpt import query_gpt
from schedule_parser import TimetableProcessor


In [2]:
processor = TimetableProcessor(database_type="sqlite")  # or "csv"
current_events = processor.format_events_as_llm_output(include_header=False)
print(current_events)

事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 编写核心模块
日期: 2025-03-03
时间段: 14:00-16:00
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 调试与测试
日期: 2025-03-03
时间段: 16:15-17:15
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 准备食材
日期: 2025-03-03
时间段: 18:30-19:00
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 烹饪
日期: 2025-03-03
时间段: 19:00-19:30
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 准备签证材料清单
日期: 2025-03-03
时间段: 20:00-21:00
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 准备会议材料
日期: 2025-03-04
时间段: 08:00-09:00
类型: 任务事项
截止日期：2025-03-04
重要程度：4

事项: 上课
日期: 2025-03-04
时间段: 11:00-17:00
类型: 固定日程
截止日期：2025-03-04
重要程度：3

事项: 选择花束
日期: 2025-03-05
时间段: 19:00-19:30
类型: 任务事项
截止日期：2025-03-05
重要程度：3

事项: 下单购买
日期: 2025-03-05
时间段: 19:30-20:00
类型: 任务事项
截止日期：2025-03-05
重要程度：3


In [3]:
import time
user_prompt = input("你有什么想放进计划表的东西吗？")
start_time = time.time()
response = query_deepseek(user_prompt, current_events, model="deepseek-chat")
#response = query_gpt(user_prompt, current_events)
end_time = time.time()
print(f"模型推理时间：{end_time - start_time}秒")
print("\n模型回复：")
print(response)

模型推理时间：17.295809268951416秒

模型回复：
新任务：帮我把后天买花时间提早一个小时
帮我把后天买花时间提早一个小时 → 拆分为：
- 选择花束（预计需30分钟，时段：2025-03-05 18:00-18:30）
- 下单购买（预计需30分钟，时段：2025-03-05 18:30-19:00）


日程建议：
事项: 选择花束
日期: 2025-03-05
时间段: 18:00-18:30
类型: 任务事项
截止日期：2025-03-05
重要程度：3
变动：更改

事项: 下单购买
日期: 2025-03-05
时间段: 18:30-19:00
类型: 任务事项
截止日期：2025-03-05
重要程度：3
变动：更改


In [4]:
# Process the response and update the database
summary = processor.process_events(response)
print(summary)

# This gets all the events in the database
formatted_output = processor.format_events_as_llm_output()
print(formatted_output)

# This gets all the events for a specific date
today_events = processor.get_events_for_date("2025-03-03")
today_formatted = processor.format_events_as_llm_output(today_events)
#print(today_formatted)


Extracted 2 events from LLM output
Event 1: 选择花束 - 2025-03-05 - 更改
Event 2: 下单购买 - 2025-03-05 - 更改
{'added': 0, 'modified': 2, 'deleted': 0, 'unchanged': 0, 'skipped': 0, 'errors': [], 'warnings': []}
事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 编写核心模块
日期: 2025-03-03
时间段: 14:00-16:00
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 调试与测试
日期: 2025-03-03
时间段: 16:15-17:15
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 准备食材
日期: 2025-03-03
时间段: 18:30-19:00
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 烹饪
日期: 2025-03-03
时间段: 19:00-19:30
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 准备签证材料清单
日期: 2025-03-03
时间段: 20:00-21:00
类型: 任务事项
截止日期：2025-03-03
重要程度：3

事项: 准备会议材料
日期: 2025-03-04
时间段: 08:00-09:00
类型: 任务事项
截止日期：2025-03-04
重要程度：4

事项: 上课
日期: 2025-03-04
时间段: 11:00-17:00
类型: 固定日程
截止日期：2025-03-04
重要程度：3

事项: 选择花束
日期: 2025-03-05
时间段: 18:00-18:30
类型: 任务事项
截止日期：2025-03-05
重要程度：3

事项: 下单购买
日期: 2025-03-05
时间段: 18:30-19:00
类型: 任务事项
截止

In [4]:
# If there are duplicate events, remove them
# I have added duplicate detection so we don't need this anymore
result = processor.remove_duplicates()
print(f"Removed {result['removed_duplicates']} duplicate events")

Removed 4 duplicate events


In [8]:
formatted_output = processor.format_events_as_llm_output()
print(formatted_output)

事项: 编写程序代码
日期: 2025-03-03
时间段: 09:00-11:00
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 测试调试
日期: 2025-03-03
时间段: 11:10-12:10
类型: 任务事项
截止日期：2025-03-03
重要程度：4

事项: 编写核心模块
日期: 2025-03-03
时间段: 14:00-16:00
类型: 任务事项
截止日期：2025-03-03
重要程度：5

事项: 调试与测试
日期: 2025-03-03
时间段: 16:15-17:15
类型: 任务事项
截止日期：2025-03-03
重要程度：5
